# Examples of R and parallel implementations

Paul Rodriguez
SDSC 8/2018

This example will set up some data and use the 'for each with dopar' 

In [1]:
# ----------------------------------------------------
#  doParallel package
#    uses 'do' or 'dopar' to run single/multicore execution
# ----------------------------------------------------
library(doParallel)
registerDoParallel(cores=24)  #register 1 to 24 nodes on comet, depending on situation

print('Start dopar ')



Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


[1] "Start dopar "


In [2]:
#Explore what variables is put into the parallel executions:
#    run this cell

dummy     = matrix(seq(1,24),24,1)  #make a dummy vector of numbers 1...24
Y_outside = matrix(0,24,1)          #make a vector outside the for loop

results = foreach(i = 1:24 ) %dopar%     #default combines results into a list  
{
  Y_inside    = 2*dummy[i]   #The 'dummy' variable is readable inside the forloop
  Y_outside[i]= Y_inside-.1     #Is the Y_outside available for assignment?  
  return(Y_inside)
}

print(results)

[[1]]
[1] 2

[[2]]
[1] 4

[[3]]
[1] 6

[[4]]
[1] 8

[[5]]
[1] 10

[[6]]
[1] 12

[[7]]
[1] 14

[[8]]
[1] 16

[[9]]
[1] 18

[[10]]
[1] 20

[[11]]
[1] 22

[[12]]
[1] 24

[[13]]
[1] 26

[[14]]
[1] 28

[[15]]
[1] 30

[[16]]
[1] 32

[[17]]
[1] 34

[[18]]
[1] 36

[[19]]
[1] 38

[[20]]
[1] 40

[[21]]
[1] 42

[[22]]
[1] 44

[[23]]
[1] 46

[[24]]
[1] 48



# Ex1  
 What is the value of Y_outside now?
 type and run: 
         str(Y_outside)  

In [3]:
#Empty cell for ex1, insert str(Y_outside)  here and run cell

# Ex2 

What was the value of Y_outside during the for loop execution?

   A. change the return statement below to return both Y_inside and Y_outside as:
            return(list(Y_inside,Y_outside[i]))   


   B. then  print(results)   and note that the output is a list of lists



In [ ]:
#For EX2

dummy     = matrix(seq(1,24),24,1)  #make a dummy vector of numbers 1...24
Y_outside = matrix(0,24,1)          #make a vector outside the for loop

results = foreach(i = 1:24 ) %dopar%     #default combines results into a list  
{
  Y_inside    = 2*dummy[i]   #The 'dummy' variable is readable inside the forloop
  Y_outside[i]= Y_inside-.1     #Is the Y_outside available for assignment?  
  return(Y_inside)
}

print(results)

In summary: doParallel can copy 'dummy' and 'Y_outside' to all cores, 
 but can not allow cores to make assignments to Y_outside outside the for loop

In other words, the cores cannot depend on what other cores might do


# EX3

Let's change how the results are combined -
    
    change the foreach statement to add a function after the loop index:
    
          results = foreach(i = 1:24, .combine=cbind) %dopar%  



In [ ]:
#For EX3

dummy     = matrix(seq(1,24),24,1)  #make a dummy vector of numbers 1...24
Y_outside = matrix(0,24,1)          #make a vector outside the for loop

results = foreach(i = 1:24 ) %dopar%     #default combines results into a list  
{
  Y_inside    = 2*dummy[i]   #The 'dummy' variable is readable inside the forloop
  Y_outside[i]= Y_inside-.1     #Is the Y_outside available for assignment?  
  return(Y_inside)
}

print(results)

# Now let's explore memory usage and parallelization

In [6]:
#clear out R workspace
ls()            #list variables
rm(list=ls())   #remove this list
print('after removal:')
ls()

[1] "Y_outside" "dummy"     "results"

[1] "after removal:"


character(0)

In [7]:
# Make up some random data and list out the object sizes
N=8000000;    #N rows
P=24;       #P columns
#N=1000000;  #1Mx1K matrix is 7.5G and crashes R and the node
#P=1000;    
X     =matrix(rnorm(N*P),N,P)
B     =seq(1,P)  #make coeffiencts as 1,2,...P
dim(X)

[1] 8000000      24

In [8]:
#Use object.size function and some format to print out 10 largest variables
sizes       = sapply(ls(), function(n) object.size(get(n)), simplify = FALSE) 
print(sapply(sizes[order(-as.integer(sizes))],   
                   function(s) format(s, unit = 'auto'))[1:10])  #print out size in more readable units

          X           B           N           P        <NA>        <NA> 
   "1.4 Gb" "168 bytes"  "48 bytes"  "48 bytes"          NA          NA 
       <NA>        <NA>        <NA>        <NA> 
         NA          NA          NA          NA 


# EX 4
  get new xterm window, ssh to this comet node,
  
     enter>  top 

 observe the total memory usage in top section


In [9]:
#Here is a loop with large matrix operation
ptm <- proc.time()

results = foreach(i = 1:24, .combine=cbind) %dopar%   
                               #combine results using 'cbind' to put into rows 
{
  Sys.sleep(10)
  Y                 = X %*% B
  vY                = var(Y)
  Sys.sleep(10)
  sizes             = sapply(ls(), function(n) object.size(get(n)), simplify = FALSE) 
  tmp               = sapply(sizes[order(-as.integer(sizes))],   
                              function(s) format(s, unit = 'auto'))[1:10]
                             #sizes[order(-as.integer(sizes))]

  return(tmp)
}


looptime=proc.time() - ptm   #returns the user,system,and elapsed time
print(looptime)

print(results[,1])
                             

   user  system elapsed 
 35.254   0.666  22.063 
          Y          vY           i        <NA>        <NA>        <NA> 
    "61 Mb" "208 bytes"  "48 bytes"          NA          NA          NA 
       <NA>        <NA>        <NA>        <NA> 
         NA          NA          NA          NA 


# EX5
Now, add the following line into the function before the 2nd sleep

  lm_result         = lm(Y~X[,i])  

and observe the peak memory usage

# EX6
Now change the foreach to use  i=1:12

Rerun the cell and look at the top output -

Does the peak memory change by 50%?

# EX7 optional
We might try to use an iterator function so that only 1 column of X is used
 execute this once
  library(iterators)  

 and put these in, replacing the corresponding statement
   results = foreach(Xc=iter(X, by='col'), .combine=cbind) %dopar%   
   lm_result         = lm(Y~Xc)  

Q; Does it alleviate memory?
Try it:

In [ ]:
#Lets compare another way;  OPTIONAL

dolm = function(Xc)
{  Sys.sleep(10)
  Y                 = Xc %*% B
  lm_result         = lm(Y~Xc)  
  Sys.sleep(10)
  sizes             = sapply(ls(), function(n) object.size(get(n)), simplify = FALSE) 
  tmp               = sapply(sizes[order(-as.integer(sizes))],   
                              function(s) format(s, unit = 'auto'))[1:10]
  return(tmp)
}

results=mclapply(X,function(Xc) dolm(Xc),mc.preschedule = TRUE, mc.cores = 24)


print(results)